In [1]:
# Libraries for data handling and simple visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Fancy progress bars
from tqdm import tqdm

# Setting pandas to display all necessary columns
pd.set_option("display.max_columns", 87)
pd.set_option("display.max_rows", 87)

# Capstone:  Data Loading, and Data Cleaning

1. [Data Loading](#Data-loading)
    - [Collecting Data](#Collecting-Data)
    - [Loading Data](#Loading-and-Merging)
2. [Data Cleaning](#Data-cleaning)
    - [Transforming Dtypes](#Transforming-dtypes)
    - [NaN Values and Outliers](#NaN-Values-and-Outliers)
    - [Droping Features](#Droping-features)
3. [Exporting](#Exporting-Clean-Data)


## Data loading

### Collecting Data

The dataset we will be working with was synthetically created by the *Canadian Institute for Cybersecurity* (CIC). Initially, the dataset, known as [CIC-IDS2017](https://www.unb.ca/cic/datasets/ids-2017.html), comprises PCAP (Packet Capture) files obtained using the [CICFlowMeter](https://github.com/ahlashkari/CICFlowMeter) tool. These PCAP files were subsequently converted into `.csv` files using the same tool.

The original dataset, however, contains various errors that may impact the predictions of traditional machine learning algorithms. Such errors include mislabeling of benign and malicious packets, as well as duplication of connections from the network. To address this, a French research team led by Maxime Lanvin developed a [cleaner version](https://hal.science/hal-03775466) of the dataset for CRiSIS 2022 (The 17th International Conference on Risks and Security of Internet and Systems). This capstone project is built upon this refined dataset, which we will further enhance and manipulate to create a machine learning model capable of detecting network intrusions.

Throughout this capstone project, our goal is to refine the dataset, perform insightful data manipulation, and develop a powerful machine learning model that can effectively identify network intrusions. By leveraging this refined dataset, we aim to contribute to the advancement of cybersecurity and enhance network security measures.

### Loading and Merging

To kickstart our data preparation process, the first step involves merging the dataset created by Maxime Lanvin and his team into a comprehensive `.csv` file. This merging enables us to efficiently handle NaN (null) values and outliers, ensuring the dataset's cleanliness and integrity. The following section outlines the process of merging the datasets, setting the stage for subsequent data cleaning procedures.

In [2]:
# Create an empty DataFrame to store the merged data
raw_data = pd.DataFrame()

# Create an array of file paths to be loaded
files = np.array(['data/Monday-WorkingHours.csv', 'data/Tuesday-WorkingHours.csv', 'data/Wednesday-WorkingHours.csv', 'data/Thursday-WorkingHours.csv', 'data/Friday-WorkingHours.csv'])

for file in tqdm(files, desc='Loading and Merging'):
    temp = pd.read_csv(file)
    raw_data = pd.concat([raw_data, temp])
    del temp

Loading and Merging: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.81s/it]


In [3]:
# Sanity Check
raw_data.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2096448 entries, 0 to 548827
Columns: 87 entries, Flow ID to Label
dtypes: float64(45), int64(37), object(5)
memory usage: 1.4+ GB


In [4]:
print(f'Our dataset contains {raw_data.shape[0]} rows and {raw_data.shape[1]} columns')

Our dataset contains 2096448 rows and 87 columns


Upon analyzing our dataset, we observe a total of **2,096,448** data entries, representing individual packets from the original PCAP files. These packets are described by **87** features. The substantial size of this dataset amounts to approximately **1.4 GB** of storage. The large data volume is a result of the extensive number of data points and the storage format used. To optimize our dataset's manageability and prevent potential kernel crashes, we should inspect the maximum and minimum values. If these values fall within the range that an *int32* or *float32* can handle, we can efficiently convert them into these data types, significantly reducing the dataset's overhead.

In [5]:
# Looking only at int data that may exceed the int32 maximum
raw_data.select_dtypes(include=[int]).max() > np.iinfo(np.int32).max

Src Port               False
Dst Port               False
Protocol               False
Flow Duration          False
Total Fwd Packet       False
Total Bwd packets      False
Fwd PSH Flags          False
Bwd PSH Flags          False
Fwd URG Flags          False
Bwd URG Flags          False
Fwd RST Flags          False
Bwd RST Flags          False
Fwd Header Length      False
Bwd Header Length      False
FIN Flag Count         False
SYN Flag Count         False
RST Flag Count         False
PSH Flag Count         False
ACK Flag Count         False
URG Flag Count         False
CWR Flag Count         False
ECE Flag Count         False
Fwd Bytes/Bulk Avg     False
Fwd Packet/Bulk Avg    False
Fwd Bulk Rate Avg      False
Bwd Bytes/Bulk Avg     False
Bwd Packet/Bulk Avg    False
Bwd Bulk Rate Avg      False
Subflow Fwd Packets    False
Subflow Fwd Bytes      False
Subflow Bwd Packets    False
Subflow Bwd Bytes      False
FWD Init Win Bytes     False
Bwd Init Win Bytes     False
Fwd Act Data P

In [6]:
# Further testing to check the behavior of the Total TCP Flow Time column
count = 0
for val in raw_data['Total TCP Flow Time'].values:
    if val > np.iinfo(np.int32).max:
        count = count + 1
print(f'The `Total TCP Flow Time` feature exceeds the int32 limit {count} times')

The `Total TCP Flow Time` feature exceeds the int32 limit 3214 times


In [7]:
raw_data['Total TCP Flow Time'].describe()

count    2.096448e+06
mean     3.260285e+07
std      3.932272e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.500000e+01
75%      3.317919e+06
max      3.029044e+10
Name: Total TCP Flow Time, dtype: float64

Based on our exploration, we've observed that among our integer features, only the `Total TCP Flow Time` column surpasses the maximum value of *int32*. Further investigation revealed that the 75th percentile of the `Total TCP Flow Time` column also exceeds the *int32* maximum, making it unsuitable for transformation without potential data loss.

However, the good news is that the remaining integer features are highly eligible for conversion into a more memory-efficient data type. By optimizing these columns, we can significantly reduce the dataset's memory footprint while preserving data integrity and analytical accuracy.

In [8]:
# Looking only at float data that may exceed the float32 maximum
raw_data.select_dtypes(include=[float]).max() > np.finfo(np.float32).max

Total Length of Fwd Packet    False
Total Length of Bwd Packet    False
Fwd Packet Length Max         False
Fwd Packet Length Min         False
Fwd Packet Length Mean        False
Fwd Packet Length Std         False
Bwd Packet Length Max         False
Bwd Packet Length Min         False
Bwd Packet Length Mean        False
Bwd Packet Length Std         False
Flow Bytes/s                   True
Flow Packets/s                 True
Flow IAT Mean                 False
Flow IAT Std                  False
Flow IAT Max                  False
Flow IAT Min                  False
Fwd IAT Total                 False
Fwd IAT Mean                  False
Fwd IAT Std                   False
Fwd IAT Max                   False
Fwd IAT Min                   False
Bwd IAT Total                 False
Bwd IAT Mean                  False
Bwd IAT Std                   False
Bwd IAT Max                   False
Bwd IAT Min                   False
Fwd Packets/s                 False
Bwd Packets/s               

In [9]:
raw_data[['Flow Bytes/s', 'Flow Packets/s']].describe()

,Flow Bytes/s,Flow Packets/s
count,2.062297e+06,2.096448e+06
mean,inf,inf
std,NaN,NaN
min,0.000000e+00,1.710381e-02
25%,1.396960e+02,3.447839e+00
50%,3.022169e+03,6.268610e+01
75%,6.701318e+04,8.733624e+03
max,inf,inf


Upon examining the float values in our dataset, we have identified the presence of outliers in the form of `inf`. After consulting the documentation from *CICFlowmeter-V4.0*, it becomes evident that these outliers stem from anomalies during the packet capture process. To maintain the integrity and reliability of our dataset, it is crucial to take appropriate action by eliminating these extreme data points.

On a positive note, we found that all other floating-point features do not present any issues, making them suitable candidates for conversion to *float32*. By making this transformation, we can effectively reduce the memory load, optimizing our dataset for more efficient processing while retaining the necessary level of precision for our analyses.

---
## Data cleaning

Even with our leaner dataset, the conventional approach of creating a separate dataset for dtype conversions and value removal may not be feasible on our current machine. As a possible next step, we should consider utilizing a cluster to distribute the computational load effectively.

By leveraging a cluster computing system, we can parallelize the data processing tasks, allowing multiple machines to work collaboratively on different portions of the dataset simultaneously. This approach significantly enhances the processing speed and efficiency, making it possible to handle large-scale data transformations and optimizations.

### Transforming dtypes

In [10]:
# Converting the appropriate features into a more memory-friendly type
integer = []
f = []
for i in raw_data.columns[:-1]:
    if raw_data[i].dtype == "int64":
        if i == 'Total TCP Flow Time':
            continue
        integer.append(i)
    elif raw_data[i].dtype == "object":
        pass
    else: 
        f.append(i)

raw_data[integer] = raw_data[integer].astype("int32")
raw_data[f] = raw_data[f].astype("float32")

In [11]:
#Sanity Check
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2096448 entries, 0 to 548827
Data columns (total 87 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Flow ID                     object 
 1   Src IP                      object 
 2   Src Port                    int32  
 3   Dst IP                      object 
 4   Dst Port                    int32  
 5   Protocol                    int32  
 6   Timestamp                   object 
 7   Flow Duration               int32  
 8   Total Fwd Packet            int32  
 9   Total Bwd packets           int32  
 10  Total Length of Fwd Packet  float32
 11  Total Length of Bwd Packet  float32
 12  Fwd Packet Length Max       float32
 13  Fwd Packet Length Min       float32
 14  Fwd Packet Length Mean      float32
 15  Fwd Packet Length Std       float32
 16  Bwd Packet Length Max       float32
 17  Bwd Packet Length Min       float32
 18  Bwd Packet Length Mean      float32
 19  Bwd Packet Length Std 

Our efforts to optimize the dataset have yielded significant results, reducing the memory usage from 1.4 GB to just 780 MB. This substantial reduction provides us with enhanced flexibility in handling the data, empowering us to perform various analyses and operations more efficiently. With a leaner dataset, we can now navigate through the data more smoothly and explore deeper insights without encountering as many memory-related constraints.

### NaN Values and Outliers

With our leaner dataset at hand, we can now proceed with the implementation of more conventional cleaning techniques, focusing on addressing null and extreme values. As we've already identified outliers in the form of `inf`, our first step will involve removing these problematic rows. The documentation from the tool responsible for capturing the packets from the network indicates that these inf values resulted from recording errors. Therefore to avoid adding unreliable data for our model to learn from we should remove it.

In [12]:
# Checking for null values before removing anything else
raw_data.isna().sum()

Flow ID                           0
Src IP                            0
Src Port                          0
Dst IP                            0
Dst Port                          0
Protocol                          0
Timestamp                         0
Flow Duration                     0
Total Fwd Packet                  0
Total Bwd packets                 0
Total Length of Fwd Packet        0
Total Length of Bwd Packet        0
Fwd Packet Length Max             0
Fwd Packet Length Min             0
Fwd Packet Length Mean            0
Fwd Packet Length Std             0
Bwd Packet Length Max             0
Bwd Packet Length Min             0
Bwd Packet Length Mean            0
Bwd Packet Length Std             0
Flow Bytes/s                  34151
Flow Packets/s                    0
Flow IAT Mean                 42998
Flow IAT Std                  42998
Flow IAT Max                  42998
Flow IAT Min                  42998
Fwd IAT Total                     0
Fwd IAT Mean                

In [13]:
# Removing infinity values from our data
raw_data = raw_data[~raw_data.isin([np.inf, -np.inf]).any(axis=1)]

In [14]:
# Checking for null values
raw_data.isna().sum()

Flow ID                       0
Src IP                        0
Src Port                      0
Dst IP                        0
Dst Port                      0
Protocol                      0
Timestamp                     0
Flow Duration                 0
Total Fwd Packet              0
Total Bwd packets             0
Total Length of Fwd Packet    0
Total Length of Bwd Packet    0
Fwd Packet Length Max         0
Fwd Packet Length Min         0
Fwd Packet Length Mean        0
Fwd Packet Length Std         0
Bwd Packet Length Max         0
Bwd Packet Length Min         0
Bwd Packet Length Mean        0
Bwd Packet Length Std         0
Flow Bytes/s                  0
Flow Packets/s                0
Flow IAT Mean                 0
Flow IAT Std                  0
Flow IAT Max                  0
Flow IAT Min                  0
Fwd IAT Total                 0
Fwd IAT Mean                  0
Fwd IAT Std                   0
Fwd IAT Max                   0
Fwd IAT Min                   0
Bwd IAT 

Fortunately, it appears that all of our null values align with the recording errors during the packet capture process. Consequently, by removing the rows containing inf values, we have successfully eliminated all the NaN (null) values from our dataset as well. This stroke of luck simplifies our data cleaning process, as we no longer need to handle separate cases for null values. 

### Droping features

In the interest of refining our dataset to focus on relevant information for our model, we have decided to remove the following columns:

1. **'Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count'**: These flags pertain to a deprecated protocol and are not useful for our model's analysis. Here is a [neat writeup](https://packetlife.net/blog/2011/mar/2/tcp-flags-psh-and-urg/) that explains more about URG Flags

2. **'Flow ID', 'Src IP', 'Src Port', 'Dst IP'**: To ensure that the model focuses on discerning packet classifications and not machine-specific details, we will remove columns related to individual machine addresses.

3. **'Timestamp'**: Likewise, we aim to guide the model in identifying attack patterns rather than associating them with specific timestamps, so we will exclude this column from our analysis.

By eliminating these columns, we streamline our dataset to concentrate on essential features, enabling the model to better grasp the characteristics that differentiate benign and malignant packets.

In [15]:
# Removing columns that may misguide our model or offer no important information
drop_cols = ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp']
print(raw_data.shape)
raw_data = raw_data.drop(columns=drop_cols)

(2053039, 87)


In [16]:
# Sanity Check
raw_data.shape

(2053039, 79)

## Exporting Clean Data

As the final step in our data cleanup process, we will proceed to export this clean dataset for further use in exploratory data analysis (EDA) and modeling. By exporting the cleaned data, we create a reliable and standardized foundation for subsequent analyses and model development.

In [17]:
# Save the cleaned data to a CSV file
raw_data.to_csv('data/WorkWeek_CleanData.csv', encoding='utf-8',index=False)

In [18]:
#Importing json to export the appropiate dtyes to avoid convertions on other notebooks
import json
# Create the dtypes dictionary from df.dtypes
dtypes_dict = raw_data.dtypes.apply(lambda x: x.name).to_dict()

# Save the dtypes to a JSON file using the json library
with open('data/dtypes.json', 'w') as json_file:
    json.dump(dtypes_dict, json_file)

In [19]:
#Retrieving the used memory space (somewhat unecessary)
del raw_data